## Examples of plotting maps and GIS data

This notebook is a **WIP** and the examples below do not work out-of-the-box on MSI systems at present.

`cartopy` is not installed in the MSI-provided Python environments, so you will need to provide your own suitable environment using Conda or pip. 

## Plotting geographic information with Cartopy

First we load some additional modules:

In [1]:
import datetime
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

ModuleNotFoundError: No module named 'cartopy'

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.stock_img()

ny_lon, ny_lat = -75, 43
delhi_lon, delhi_lat = 77.23, 28.61

plt.plot([ny_lon, delhi_lon], [ny_lat, delhi_lat],
         color='blue', linewidth=2, marker='o',
         transform=ccrs.Geodetic(),
         )

plt.plot([ny_lon, delhi_lon], [ny_lat, delhi_lat],
         color='gray', linestyle='--',
         transform=ccrs.PlateCarree(),
         )

plt.text(ny_lon - 3, ny_lat - 12, 'New York',
         horizontalalignment='right',
         transform=ccrs.Geodetic())

plt.text(delhi_lon + 3, delhi_lat - 12, 'Delhi',
         horizontalalignment='left',
         transform=ccrs.Geodetic())

plt.show()

In [ ]:
import cartopy.feature

In [ ]:
import os
import matplotlib.pyplot as plt
from netCDF4 import Dataset as netcdf_dataset
import numpy as np

from cartopy import config
import cartopy.crs as ccrs


# get the path of the file. It can be found in the repo data directory.
fname = os.path.join(config["repo_data_dir"],
                     'netcdf', 'HadISST1_SST_update.nc'
                     )

dataset = netcdf_dataset(fname)
sst = dataset.variables['sst'][0, :, :]
lats = dataset.variables['lat'][:]
lons = dataset.variables['lon'][:]

ax = plt.axes(projection=ccrs.PlateCarree())

plt.contourf(lons, lats, sst, 60,
             transform=ccrs.PlateCarree())

ax.coastlines()

plt.show()


Basemap is deprecated in favor of Cartopy - below included for reference only

Plotting geographic information with Basemap

This example is based on ``plotsst.py`` from the [Basemap examples page](http://matplotlib.org/basemap/users/examples.html).

First we load some additional modules:

In [ ]:
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset, date2index
from datetime import datetime

A simple example: plot a world map with the current day/night shaded

In [ ]:
# miller projection 
map = Basemap(projection='mill',lon_0=180)
# plot coastlines, draw label meridians and parallels.
map.drawcoastlines()
map.drawparallels(np.arange(-90,90,30),labels=[1,0,0,0])
map.drawmeridians(np.arange(map.lonmin,map.lonmax+30,60),labels=[0,0,0,1])
# fill continents 'coral' (with zorder=0), color wet areas 'aqua'
map.drawmapboundary(fill_color='aqua')
map.fillcontinents(color='coral',lake_color='aqua')
# shade the night areas, with alpha transparency so the 
# map shows through. Use current time in UTC.
date = datetime.utcnow()
CS=map.nightshade(date)
plt.title('Day/Night Map for %s (UTC)' % date.strftime("%d %b %Y %H:%M:%S"))
plt.show()

Now for a more complex example.

Here we read in the sea surface temps and ice from NOAA. See how easy it is to fetch external datasets!

In [ ]:
date = datetime( 2017,10,1,0) # date to plot.
# open dataset.
dataset = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.oisst.v2/sst.wkmean.1990-present.nc')
icedata = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.oisst.v2/icec.wkmean.1990-present.nc')
timevar = dataset.variables['time']
timeindex = date2index(date,timevar) # find time index for desired date.

Create some variables. Read ``sst`` and ``ice`` from the dataset we just downloaded, and make a grid of latitude and longitude.

In [ ]:
# read sst.  Will automatically create a masked array using
# missing_value variable attribute. 'squeeze out' singleton dimensions.
sst = dataset.variables['sst'][timeindex,:].squeeze()
# read ice.
ice = icedata.variables['icec'][timeindex,:].squeeze()
# read lats and lons (representing centers of grid boxes).
lats = dataset.variables['lat'][:]
lons = dataset.variables['lon'][:]
lons, lats = np.meshgrid(lons,lats)

Some fiddling since the format of the land and ice masks changed since we wrote this example.

In [ ]:
land = ice.copy()
land.data[:] = 0
land.mask = ~land.mask[:]
ice.mask[np.where(ice.data == 0)] = True

Next we set up a figure and a map projection

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0.05,0.05,0.9,0.9])
# create Basemap instance without coastlines
m = Basemap(projection='kav7',lon_0=0,resolution=None)
m.drawmapboundary(fill_color='0.3')

Finally, we plot the data onto our world map.

In [ ]:
# plot sst, then ice with pcolor
im1 = m.pcolormesh(lons,lats,sst,shading='flat',cmap=plt.cm.hot,latlon=True)
im2 = m.pcolormesh(lons,lats,ice,shading='flat',cmap=plt.cm.gist_rainbow,latlon=True)
im3 = m.pcolormesh(lons,lats,land,shading='flat',cmap=plt.cm.hot,latlon=True)
# draw parallels and meridians, but don't bother labelling them.
m.drawparallels(np.arange(-90.,99.,30.))
m.drawmeridians(np.arange(-180.,180.,60.))
# add colorbar
cb = m.colorbar(im1,"bottom", size="5%", pad="2%")
# add a title.
plt.title('SST and ICE analysis for %s'%date)
plt.show()